# [VAE in Pyro](http://pyro.ai/examples/vae.html#VAE-in-Pyro)

Let’s see how we implement a VAE in Pyro. The dataset we’re going to model is MNIST, a collection of images of handwritten digits. Since this is a popular benchmark dataset, we can make use of PyTorch’s convenient data loader functionalities to reduce the amount of boilerplate code we need to write:

In [ ]:
import torchvision

In [ ]:
import torchvision.transforms as transforms

In [ ]:
from torchvision.datasets import MNIST

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.environ['HOME']

In [ ]:
pixel_path = os.path.join(os.environ['HOME']+'/mnist/')

In [ ]:
pixel_path

In [ ]:
if not os.path.exists(pixel_path):
    os.mkdir(pixel_path)

In [ ]:
train_set = MNIST(root=pixel_path, train=True, transform=transforms.ToTensor(), download=True)

In [27]:
test_set = MNIST(root=pixel_path, train=False, transform=transforms.ToTensor())